In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
import os
import glob
import time
import shutil
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms

In [2]:
DATA_FOLDER = './data'
BATCH_SIZE = 4

filenames = sorted(os.listdir(DATA_FOLDER))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# Show labels of every class
with open(os.path.join(DATA_FOLDER, 'labels.txt'), "r") as f:
    classes_text = f.read()
classes = classes_text.split('\n')
print(classes_text)

1 гривня (зразка 2006 р)
1 гривня (зразка 2006 р, зворотнiй бiк)
2 гривнi (зразка 2004 р)
2 гривнi (зразка 2004 р, зворотнiй бiк)
5 гривень (зразка 2004 р)
5 гривень (зразка 2004 р, зворотнiй бiк)
10 гривень (зразка 2004 р)
10 гривень (зразка 2004 р, зворотнiй бiк)
20 гривень (зразка 2003 р)
20 гривень (зразка 2003 р, зворотнiй бiк)
50 гривень (зразка 2004 р)
50 гривень (зразка 2004 р, зворотнiй бiк)
100 гривень (зразка 2005 р)
100 гривень (зразка 2005 р, зворотнiй бiк)
200 гривень (зразка 2007 р)
200 гривень (зразка 2007 р, зворотнiй бiк)
500 гривень (зразка 2006 р)
500 гривень (зразка 2006 р, зворотнiй бiк)
1000 гривень (зразка 2019 р)
1000 гривень (зразка 2019 р, зворотнiй бiк)
500 гривень (зразка 2015 р)
500 гривень (зразка 2015 р, зворотнiй бiк)
100 гривень (зразка 2014 р)
100 гривень (зразка 2014 р, зворотнiй бiк)
20 гривень (зразка 2018 р)
20 гривень (зразка 2018 р, зворотнiй бiк)
50 гривень (зразка 2019 р)
50 гривень (зразка 2019 р, зворотнiй бiк)
200 гривень (зразка 2019 р)
20

In [4]:
# Gathering paths of every image in dataset.
data = []

filenames = [fname for fname in filenames if fname != 'labels.txt']
for fname in filenames:
    images = glob.glob(os.path.join(DATA_FOLDER, fname, '*.png'))
    images += glob.glob(os.path.join(DATA_FOLDER, fname, '*.jpg'))
    images += glob.glob(os.path.join(DATA_FOLDER, fname, '*.jpeg'))
    data.append((fname, images))

print(data[0][0], data[0][1][0])

01-1a ./data/01-1a/tf-1565952427481.png


In [5]:
# Splitting all the images into train, dev, test sets.

train_dir = './train'
dev_dir = './dev'
test_dir = './test'
random.seed(42)

if not os.path.isdir(train_dir) or not os.path.isdir(dev_dir) or not os.path.isdir(test_dir):

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(dev_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    
    for name, images in data:
        os.makedirs(os.path.join(train_dir, name))
        os.makedirs(os.path.join(dev_dir, name))
        os.makedirs(os.path.join(test_dir, name))
        
        train_end = int(0.8 * len(images))
        dev_end = int(0.9 * len(images))
        
        random.shuffle(images)
        
        train = images[:train_end]
        dev = images[train_end:dev_end]
        test = images[dev_end:]
        
        # Copy images into corresponding directories.
        for image_dir, images_in_dir in zip([train_dir, dev_dir, test_dir], [train, dev, test]):
            for image in images_in_dir:
                dst = os.path.join(os.path.join(image_dir, name), os.path.basename(image))
                shutil.copyfile(image, dst)

In [6]:
transf = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

# Creating datasets for every folder.
train_folder = ImageFolder(root=train_dir, transform=transf)
dev_folder = ImageFolder(root=dev_dir, transform=transf)
test_folder = ImageFolder(root=test_dir, transform=transf)

# Creating loaders for every dataset.
train_loader = DataLoader(train_folder, batch_size=BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(dev_folder, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_folder, batch_size=BATCH_SIZE, shuffle=True)

# Showing example of coded labels.
images, labels = iter(train_loader).next()
print(labels)

tensor([11, 16, 19, 30])


In [12]:
# Simple convolution neural network (given by pytorch tutorial)

class Baseline(nn.Module):
    def __init__(self):
        super(Baseline, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(16 * 56 * 56, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 33)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [13]:
model = Baseline()  # Choose your fighter.
model.to(device)  # For GPU

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [14]:
for epoch in range(20):  

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 0.330030
[1,   400] loss: 0.214296
[1,   600] loss: 0.142898
[1,   800] loss: 0.113871
[1,  1000] loss: 0.089746
[2,   200] loss: 0.052259
[2,   400] loss: 0.053182
[2,   600] loss: 0.046201
[2,   800] loss: 0.044364
[2,  1000] loss: 0.033695
[3,   200] loss: 0.028485
[3,   400] loss: 0.035901
[3,   600] loss: 0.029149
[3,   800] loss: 0.036325
[3,  1000] loss: 0.028780
[4,   200] loss: 0.022520
[4,   400] loss: 0.023575
[4,   600] loss: 0.028341
[4,   800] loss: 0.018719
[4,  1000] loss: 0.033137
[5,   200] loss: 0.016662
[5,   400] loss: 0.024980
[5,   600] loss: 0.018197
[5,   800] loss: 0.017332
[5,  1000] loss: 0.018291
[6,   200] loss: 0.011364
[6,   400] loss: 0.018000
[6,   600] loss: 0.019468
[6,   800] loss: 0.032790
[6,  1000] loss: 0.024177
[7,   200] loss: 0.015582
[7,   400] loss: 0.009650
[7,   600] loss: 0.018916
[7,   800] loss: 0.010132
[7,  1000] loss: 0.008131
[8,   200] loss: 0.014259
[8,   400] loss: 0.012511
[8,   600] loss: 0.005195
[8,   800] l

### Saving trained model

In [15]:
MODEL_PATH = './trained_models'
os.makedirs(MODEL_PATH, exist_ok=True)
str_time = time.strftime("%Y-%m-%d_%H:%M_model.pth", time.gmtime())
torch.save(model.state_dict(), os.path.join(MODEL_PATH, str_time))

---

### Loading saved model

In [16]:
model = Baseline()
model.to(device)  # For GPU
model.load_state_dict(torch.load(os.path.join(MODEL_PATH, str_time)))

<All keys matched successfully>

In [17]:
# Testing accuracy

correct = 0
total = 0
with torch.no_grad():
    for data in dev_loader:
        images, labels = data[0].to(device), data[1].to(device)  # For GPU
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the dev images: %d %%' % (100 * correct / total))

Accuracy on the dev images: 93 %


In [18]:
# Testing accuracy for each class

class_correct = [0.0 for i in range(33)]
class_total = [0.0 for i in range(33)]
with torch.no_grad():
    for data in dev_loader:
        images, labels = data[0].to(device), data[1].to(device)  # For GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(BATCH_SIZE):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(33):
    print('Accuracy of %43s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of                    1 гривня (зразка 2006 р) : 92 %
Accuracy of     1 гривня (зразка 2006 р, зворотнiй бiк) : 85 %
Accuracy of                    2 гривнi (зразка 2004 р) : 100 %
Accuracy of     2 гривнi (зразка 2004 р, зворотнiй бiк) : 100 %
Accuracy of                   5 гривень (зразка 2004 р) : 100 %
Accuracy of    5 гривень (зразка 2004 р, зворотнiй бiк) : 93 %
Accuracy of                  10 гривень (зразка 2004 р) : 93 %
Accuracy of   10 гривень (зразка 2004 р, зворотнiй бiк) : 93 %
Accuracy of                  20 гривень (зразка 2003 р) : 87 %
Accuracy of   20 гривень (зразка 2003 р, зворотнiй бiк) : 82 %
Accuracy of                  50 гривень (зразка 2004 р) : 94 %
Accuracy of   50 гривень (зразка 2004 р, зворотнiй бiк) : 100 %
Accuracy of                 100 гривень (зразка 2005 р) : 91 %
Accuracy of  100 гривень (зразка 2005 р, зворотнiй бiк) : 81 %
Accuracy of                 200 гривень (зразка 2007 р) : 93 %
Accuracy of  200 гривень (зразка 2007 р, зворотнiй 